# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [ ]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (D - A) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
def jacobi(A, b, n_iter):
  diag_1d = np.diag(A)
  B = -A.copy()
  np.fill_diagonal(B, 0)
  D = np.diag(diag_1d)
  invD = np.diag(1./diag_1d)
  BB = invD @ B 
  c = invD @ b
  x0 = np.ones(n)
  x = x0
  for _ in range(n_iter):
    x = BB @ x + c
  print("Norm of B:",np.linalg.norm(BB))
  return x

In [ ]:
n_iter=50
x = jacobi(A, b,n_iter)
print(x)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

Norm of B: 0.36436161983015336
[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


### What happens if the diagonal elements of A were smaller?

In [ ]:
rndm = np.random.RandomState(1234)
n = 10
for i in range(10,0,-1):
  A = rndm.uniform(size=(n, n)) + np.diagflat([i]*n)
  b = rndm.uniform(size=n)
  x=jacobi(A,b,n_iter)
  print("Convergence:\n",A@x-b,"\n\n")

Norm of B: 0.5388887887486234
Convergence:
 [0.00000000e+00 0.00000000e+00 1.11022302e-16 1.11022302e-16
 0.00000000e+00 5.55111512e-17 3.81639165e-17 1.11022302e-16
 5.55111512e-17 0.00000000e+00] 


Norm of B: 0.5976303078076515
Convergence:
 [2.76167977e-15 2.10942375e-15 2.10942375e-15 2.10942375e-15
 3.33066907e-15 3.66373598e-15 2.99760217e-15 2.33146835e-15
 2.22044605e-15 3.10862447e-15] 


Norm of B: 0.6105347157974735
Convergence:
 [1.47659662e-14 1.90958360e-14 1.04360964e-14 1.92068583e-14
 2.16493490e-14 1.52100554e-14 1.50990331e-14 1.97342143e-14
 1.63757896e-14 1.43357548e-14] 


Norm of B: 0.8175632200040883
Convergence:
 [7.20216191e-08 6.52711910e-08 7.90350859e-08 6.87843351e-08
 6.45559779e-08 6.61882123e-08 6.62691958e-08 6.53249093e-08
 8.06978616e-08 7.86017768e-08] 


Norm of B: 0.8027935795980838
Convergence:
 [3.92826872e-09 5.48601087e-09 4.13041590e-09 3.51212798e-09
 4.74419221e-09 4.60755495e-09 4.86854779e-09 2.98138356e-09
 5.63267100e-09 5.06950781e-09

**We can see that, for matrices with smaller values of D (diag(A)), the norm of B is higher and there's less convergence, meaning that we would need a higher number of iterations.**

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
def seidel(A,b,tolerance,max_iterations):
  x=np.zeros_like(b, dtype=np.double)
  print("Norm of A:",np.linalg.norm(A))
  #Iterate
  for k in range(max_iterations):
    x_old=x.copy()

    #Loop over rows
    for i in range (A.shape[0]):
      x[i]=(b[i]-np.dot(A[i,:i], x[:i])-np.dot(A[i,(i+1):],x_old[(i+1):]))/A[i,i]

    #Stop condition
    if np.linalg.norm(x-x_old,ord=np.inf)/np.linalg.norm(x,ord=np.inf) <= tolerance:
      break
    return x


In [ ]:
max_iterations=10000
tolerance=1e-16
n = 10

A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

x = seidel(A,b,tolerance,max_iterations)
print(x)
print("Convergence:\n",A@x-b,"\n\n")


Norm of A: 49.307661452451555
[0.0102176  0.06129089 0.04342757 0.05957914 0.00609808 0.04610898
 0.04862146 0.01027937 0.00352014 0.01049445]
Convergence:
 [0.13352649 0.08153478 0.12907675 0.05645164 0.05005403 0.01575324
 0.01366717 0.00194138 0.00324061 0.        ] 




In [ ]:
n = 10
max_iterations=10000
tolerance=1e-16
for i in range(10,0,-1):
  A = rndm.uniform(size=(n, n)) + np.diagflat([i]*n)
  b = rndm.uniform(size=n)
  x = seidel(A,b,tolerance,max_iterations)
  print("Convergence:\n",A@x-b,"\n\n")

Norm of A: 33.45255896450867
Convergence:
 [ 0.21538583  0.18047791  0.10706228  0.04134411  0.05695759  0.01263516
  0.03602039  0.02091509 -0.00278143  0.        ] 


Norm of A: 30.712695413289794
Convergence:
 [ 6.89833245e-02  2.48702411e-01  1.18936196e-01  1.27363865e-01
  1.20763054e-01  3.43904179e-02  4.46915527e-03 -2.74676619e-03
  6.27005739e-02  1.11022302e-16] 


Norm of A: 27.22381308661258
Convergence:
 [ 2.05719368e-01  2.06837931e-01  1.68398048e-01  1.41000854e-01
  1.13479409e-01  6.99305697e-02  1.10861807e-01  2.08200377e-02
  8.69393113e-03 -2.77555756e-17] 


Norm of A: 24.338023833105698
Convergence:
 [0.36910393 0.11759764 0.140503   0.2936767  0.10179393 0.05823808
 0.05438756 0.0457473  0.08545626 0.        ] 


Norm of A: 21.61812818384426
Convergence:
 [0.25917831 0.22768759 0.26348329 0.10466201 0.09185164 0.05579758
 0.00652468 0.01900446 0.03260943 0.        ] 


Norm of A: 18.373345030659866
Convergence:
 [ 4.22051068e-01  3.28514662e-01  3.00374806e-0

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
def minimum_residual(A, b, max_iterations, tol):
    print("Norm of A:",np.linalg.norm(A))
    n = A.shape[0]
    x0 = np.ones(b.shape[0])
    x = x0.copy()  # Initial solution guess
    residual_norms = []  # Track norm of the residual at each iteration
    deviation_norms = []  # Track deviation from ground truth solution at each iteration

    for iteration in range(max_iterations):
        residual = b - np.dot(A, x)  # Calculate the residual
        residual_norm = np.linalg.norm(residual)  # Compute the norm of the residual
        residual_norms.append(residual_norm)

        if residual_norm < tol:
            break  # Convergence criterion met, exit the loop

        # Calculate the iteration parameter that minimizes the residual
        alpha = np.dot(residual, np.dot(A, residual)) / np.dot(np.dot(A, residual), np.dot(A, residual))
        
        x = x + alpha * residual  # Update the solution estimate

    return x

In [ ]:
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)
max_iterations=1000
tolerance=1e-14

x = minimum_residual(A,b,max_iterations,tolerance)
A@x-b

Norm of A: 49.303030903788525


array([ 5.55111512e-16,  7.77156117e-16,  3.33066907e-16, -4.44089210e-16,
        2.22044605e-16,  7.77156117e-16,  3.74700271e-16,  5.55111512e-16,
       -3.33066907e-16,  7.21644966e-16])

In [ ]:
n = 10
max_iterations=100000
tolerance=1e-14
for i in range(10,0,-1):
  A = rndm.uniform(size=(n, n)) + np.diagflat([i]*n)
  b = rndm.uniform(size=n)
  x = minimum_residual(A,b,max_iterations,tolerance)
  print("Convergence:\n",A@x-b,"\n\n")

Norm of A: 34.05585368359852
Convergence:
 [6.66133815e-16 3.33066907e-16 7.77156117e-16 7.21644966e-16
 2.22044605e-16 4.99600361e-16 2.22044605e-16 2.22044605e-16
 4.44089210e-16 2.77555756e-16] 


Norm of A: 30.33925946989076
Convergence:
 [-4.44089210e-16 -1.38777878e-15  1.11022302e-15 -2.22044605e-16
 -2.02615702e-15 -4.44089210e-16 -1.44328993e-15  2.22044605e-16
  1.11022302e-15 -1.44328993e-15] 


Norm of A: 27.391597169886182
Convergence:
 [-3.33066907e-16 -9.43689571e-16 -3.33066907e-16 -1.16573418e-15
  5.55111512e-16 -9.99200722e-16 -1.22124533e-15  4.16333634e-16
 -8.88178420e-16 -1.11022302e-15] 


Norm of A: 24.127574790990643
Convergence:
 [-8.04911693e-16 -1.88737914e-15 -1.11022302e-16  2.22044605e-16
  6.66133815e-16 -1.11022302e-16 -6.66133815e-16 -1.44328993e-15
 -6.66133815e-16 -1.88737914e-15] 


Norm of A: 21.30094562350186
Convergence:
 [ 1.11022302e-15 -6.66133815e-16  3.38618023e-15  1.88737914e-15
 -3.33066907e-16  4.94049246e-15  3.38618023e-15 -6.66133815